# 5.2 Parallel dofs and Vector-types
In this tutorial we learn how NGSolve represents distributed finite element spaces.

In [ ]:
from ipyparallel import Cluster
c = await Cluster(engines="mpi").start_and_connect(n=4, activate=True)

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square
comm = MPI.COMM_WORLD
if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.1).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))

Create the space on the distributed mesh. All processes agree on the global number of dofs. Locally, each rank has access only to the subset of dofs associated with its elements. Some dofs are shared by several ranks:

In [ ]:
%%px
fes = H1(mesh, order=1)
# fes = L2(mesh, order=0)
print ("global dofs =", fes.ndofglobal, ", local dofs =", fes.ndof, \
       ", sum of local dofs =", comm.allreduce(fes.ndof))

## Parallel Dofs
A ParallelDofs object maintains information how dofs are connected across the cluster. The ParallelDofs object is generated by the FESpace, which has access to the connectivity of the mesh.

In [ ]:
%%px
pardofs = fes.ParallelDofs()
for k in range(pardofs.ndoflocal):
    print ("dof", k, "is shard with ranks", list(pardofs.Dof2Proc(k)))

In [ ]:
%%px
print ("I share dofs with ranks:", list(pardofs.ExchangeProcs()))
for k in range(MPI.COMM_WORLD.size):
    print ("with rank", k, "I share dofs", list(pardofs.Proc2Dof(k)))

In [ ]:
%%px
u,v = fes.TnT()
M = BilinearForm(u*v*dx).Assemble().mat
gfu = GridFunction(fes)
gfu.Set (1)
print (gfu.vec)

We see that all values are set to 1, i.e. joint dofs have the same value. We call such a vector 'cumulated'. The matrix M is stored locally assembled, i.e. every rank has the contributions from its elements. When we multiply this matrix with a cumulated vector, every rank performs a local matrix vector product. The resulting vector is stored 'distributed', i.e. the true values are obtained by adding up rank-local contributions for joint dofs:

In [ ]:
%%px
r = M.CreateColVector()
r.data = M*gfu.vec
print (r)

This cumulated/distributed pair of vectors is prefect for computing inner products. We can compute inner products of local vectors, and sum up (i.e. reduce in MPI terms) across all ranks:

In [ ]:
%%px
print ("global ip =", InnerProduct(gfu.vec, r))
localip = InnerProduct(r.local_vec, gfu.vec.local_vec)
print ("local contribution:", localip)
print ("cumulated:", comm.allreduce(localip, MPI.SUM))